In [1]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)


C:\Users\Isabel\AppData\Local\Temp\ipykernel_14848\3175742525.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('../Fase1_datos/lista_accion.json', 'r') as fichero:
    accion = json.load(fichero)

In [3]:
len(accion)

3679

In [20]:
busqueda = accion[505:1000]
len(busqueda)

495

In [5]:
# variables donde iremos almacenando todo
info_por_peli = []
fallos = []

In [21]:
# llegar a web:
driver = webdriver.Chrome()
driver.get('https://www.imdb.com/')
driver.maximize_window()
sleep(random.randint(1,3))


# rechazar cookies
driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
sleep(random.randint(1,3))

contador = 0

# entrando en cada título
for peli in busqueda:
    lista_info = [f'{peli[0]}',]
    contador +=1
    print(f'Película: {contador}')

    try:
        
        # metemos el ID en el buscador
        try:
            driver.find_element('css selector', '#suggestion-search').send_keys(peli[0], Keys.ENTER)
            sleep(random.randint(1,3))

            # extraemos info
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text
            lista_info.append(info.split('\n'))

            info_por_peli.append(lista_info)
        except:
            fallos.append(peli)
            driver.find_element('css selector', '#error > div.error_attrib > a').click()
        
                    
    except:
        continue

Película: 1
Película: 2
Película: 3
Película: 4
Película: 5
Película: 6
Película: 7
Película: 8
Película: 9
Película: 10
Película: 11
Película: 12
Película: 13
Película: 14
Película: 15
Película: 16
Película: 17
Película: 18
Película: 19
Película: 20
Película: 21
Película: 22
Película: 23
Película: 24
Película: 25
Película: 26
Película: 27
Película: 28
Película: 29
Película: 30
Película: 31
Película: 32
Película: 33
Película: 34
Película: 35
Película: 36
Película: 37
Película: 38
Película: 39
Película: 40
Película: 41
Película: 42
Película: 43
Película: 44
Película: 45
Película: 46
Película: 47
Película: 48
Película: 49
Película: 50
Película: 51
Película: 52
Película: 53
Película: 54
Película: 55
Película: 56
Película: 57
Película: 58
Película: 59
Película: 60
Película: 61
Película: 62
Película: 63
Película: 64
Película: 65
Película: 66
Película: 67
Película: 68
Película: 69
Película: 70
Película: 71
Película: 72
Película: 73
Película: 74
Película: 75
Película: 76
Película: 77
Película

In [22]:
len(info_por_peli)

989

In [23]:
len(fallos)

4

In [24]:
with open('FallosComedia.json', 'w') as fichero:
    json.dump(fallos, fichero)

In [25]:
lista_tuplas = []

In [26]:
for peli in info_por_peli:
    
    patron = re.compile(r'([a-z])([A-Z])')
    patron_duracion = re.compile(r'\b(?:\d+h\s*)?\d+min\b')
    tupla_datos = [peli[0],] 

    # listas intermedias para ir checkeando
    direccion = []
    guion = []
    nota = []
    duracion = []
    argumento = []
        
    for indice, linea in enumerate(peli[1]):
               
        
        # directores    
        if 'Dirección' in linea:
                                
            directores = patron.sub(r'\1, \2', peli[1][indice+1])
            direccion.append(directores)            


        # guionistas
        elif 'Guión' in linea:
            
            guionistas = patron.sub(r'\1, \2', peli[1][indice+1])
            guion.append(guionistas)
                
        
        # nota y duracion
        elif 'PUNTUACIÓN EN IMDb' in linea:
            nota.append(float(peli[1][indice+1].replace(',', '.')))
            
            if r'\d+h' or r'\d+min' in peli[1][indice-1]:  
                duracion.append(peli[1][indice-1])
            
                    
        # argumento
        elif 'Añade un argumento en tu idioma' in linea:
            argumento.append(peli[1][indice+1])

    # añadimos a la tupla los datos de la lista intermedia
    for elemento in [direccion, guion, nota, duracion, argumento]:
        if elemento != []:
            tupla_datos.append(elemento[0])
        else:
            tupla_datos.append(np.nan)
    
    lista_tuplas.append(tupla_datos)

In [27]:
len(lista_tuplas)

989

In [19]:
with open('AccionWIP.json', 'w') as fichero:
    json.dump(lista_tuplas, fichero)

In [10]:
len(fallos)

0

selenium - actores

In [2]:
with open('AccionWIP.json', 'r') as fichero:
    action = json.load(fichero)

In [3]:
busqueda = action

In [8]:
lista_url = []
lista_fichas = []

for pelicula in busqueda:
    # sacar urls
    lista_url.append(f'https://www.imdb.com/title/{pelicula[0]}/fullcredits/?ref_=tt_ql_1')


In [9]:
len(lista_url)

989

In [ ]:
 # sacar bloque de info
    ficha_por_peli = [pelicula[0]]
    driver.get(f'https://www.imdb.com/title/{pelicula[0]}/fullcredits/?ref_=tt_ql_1')
    info_tecnica = driver.find_element('css selector', '#main > div.article.listo').text.split('\n')
    ficha_por_peli.append(info_tecnica)

    lista_fichas.append(ficha_por_peli)

In [10]:

lista_url

['https://www.imdb.com/title/tt0113092/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0116391/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0116628/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0119273/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0120755/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0120913/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0120903/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0132910/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0134847/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0141926/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0143344/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0144964/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0144608/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0147486/fullcredits/?ref_=tt_ql_1',
 'https://www.imdb.com/title/tt0151150/fullcredits/?ref_=tt_ql

In [ ]:
<a href="/name/nm0001002/?ref_=ttfc_fc_cl_t1"> Dean Cain
</a>

In [11]:

for url in lista_url[:2]:
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    driver.find_element('css selector', '#imdb-bmo-footer > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()

    
    sleep(1)

    for actor in range(2, 12):
        
        tupla_actor = []

        xpath_actor = driver.find_element('xpath', '//*[@id="fullcredits_content"]/table[3]/tbody/tr[2]/td[2]/a')
        href_actor = xpath_actor.get_attribute('href')
        tupla_actor.append(href_actor)

        # click en actor desde página de cast
        driver.find_element('css selector', f'#fullcredits_content > table.cast_list > tbody > tr:nth-child({actor}) > td:nth-child(2) > a').click()
                    
        sleep(2)

        # extrae datos del actor
        nombre_actor = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div > h1 > span.hero__primary-text').text
        tupla_actor.append(nombre_actor)


        try:            
            datos_personales = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(20)').text.split('\n')
            tupla_actor.append(datos_personales)

        except:
            pass


        try:
            conocido_por = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1)').text.split('\n')
            tupla_actor.append(conocido_por)

        except:
            pass


        try:
            roles = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(2)').text.split('\n')
            tupla_actor.append(roles)
        except:
            pass
        
        try:
            driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__icon-link > svg').click()
            premios = driver.find_element('css selector', '#ipc-wrap-background-id').text.split('\n')
            tupla_actor.append(premios)
            driver.back()
        
        except:
            pass

        # para atrás
        driver.back()
        break
    break
    

In [12]:
tupla_actor

['https://www.imdb.com/name/nm0001002/?ref_=ttfc_fc_cl_t1',
 'Dean Cain',
 ['Datos personales',
  'Editar',
  'Sitios oficiales',
  'CameoInstagram',
  'Nombre alternativo',
  'Chief Patterson',
  'Altura',
  '1,82 m',
  'Nacimiento',
  '31 de julio de 1966Mt. Clemens, Michigan, Estados Unidos',
  'Niños',
  'Christopher Dean Cain',
  'Padres',
  'Roger Tanaka',
  'Familiares',
  'Roger Cain(Sibling)',
  'Otras obras',
  'Appeared as a motorcycle rider in George Strait\'s music video of his song "Carrying Your Love With Me"',
  'Listings de publicidad',
  '3 entrevistas7 artículos3 ilustraciones25 fotos de portada de revista'],
 ['Conocido/a por:',
  "God's Not Dead",
  '4,6',
  'Marc Shelley',
  '2014',
  'A contrarreloj',
  '6,5',
  'Chris Harrison',
  '2003',
  'Ratas a la carrera',
  '6,5',
  'Shawn Kent',
  '2001',
  'Lois y Clark: Las nuevas aventuras de Superman',
  '6,7',
  'Serie de TV',
  'BarryBill RobertsonClark Kent',
  '1993–1997 • 87 episodios'],
 ['Créditos',
  'Editar'

In [38]:
nombre_actor

'Dean Cain'

In [39]:
conocido_por

['Conocido/a por:',
 "God's Not Dead",
 '4,6',
 'Marc Shelley',
 '2014',
 'A contrarreloj',
 '6,5',
 'Chris Harrison',
 '2003',
 'Ratas a la carrera',
 '6,5',
 'Shawn Kent',
 '2001',
 'Lois y Clark: Las nuevas aventuras de Superman',
 '6,7',
 'Serie de TV',
 'BarryBill RobertsonClark Kent',
 '1993–1997 • 87 episodios']

In [36]:
premios

['']

In [40]:
datos_personales

['Datos personales',
 'Editar',
 'Sitios oficiales',
 'CameoInstagram',
 'Nombre alternativo',
 'Chief Patterson',
 'Altura',
 '1,82 m',
 'Nacimiento',
 '31 de julio de 1966Mt. Clemens, Michigan, Estados Unidos',
 'Niños',
 'Christopher Dean Cain',
 'Padres',
 'Roger Tanaka',
 'Familiares',
 'Roger Cain(Sibling)',
 'Otras obras',
 'Appeared as a motorcycle rider in George Strait\'s music video of his song "Carrying Your Love With Me"',
 'Listings de publicidad',
 '3 entrevistas7 artículos3 ilustraciones25 fotos de portada de revista']

In [41]:
roles

['Créditos',
 'Editar',
 'Reparto209',
 'Producción17',
 'Guion6',
 'Dirección3',
 'Banda sonora1',
 'Como sí mismos210',
 'Agradecimientos3',
 'metraje de archivo17',
 'IMDbPro',
 'Desplegar abajo',
 'Reparto',
 'Próximamente',
 '11',
 'Anterior',
 '198',
 'The Phantom Warrior',
 '4,8',
 'The Sheriff',
 '2024',
 "Hank's Christmas Wish",
 '5,3',
 'Mayor Dean',
 '2023',
 'The Curse',
 '7,2',
 'Serie de TV',
 'Mark Rose',
 '2023',
 '1 episodio',
 'Bumbling Ballerina',
 '4,7',
 'Detective',
 '2023',
 'Not for Sale',
 '5,0',
 'Det. Derrick Tanner',
 '2023',
 'R.A.D.A.R.: The Adventures of the Bionic Dog',
 '4,8',
 'Mayor',
 '2023',
 'Condition of Return',
 '7,1',
 'Dr. Donald Thomas',
 '2023',
 'Ghost Trader',
 '4,5',
 'Dirty Government Official',
 '2023',
 'Casting Stones',
 '3,8',
 'Pastor McGovern',
 '2023',
 'Miracle at Manchester',
 '5,7',
 'Dr. Getty',
 '2022',
 'Drop the Beat',
 '2,7',
 'Pastor',
 '2022',
 'Glass Walls',
 '3,7',
 'Ozzy',
 '2022',
 'Demon Pit',
 '4,0',
 'Roshan',
 '2

limpiar datos de tupla_actor

In [14]:
for dato in tupla_actor:
    print(dato)
    print('---')


https://www.imdb.com/name/nm0001002/?ref_=ttfc_fc_cl_t1
---
Dean Cain
---
['Datos personales', 'Editar', 'Sitios oficiales', 'CameoInstagram', 'Nombre alternativo', 'Chief Patterson', 'Altura', '1,82 m', 'Nacimiento', '31 de julio de 1966Mt. Clemens, Michigan, Estados Unidos', 'Niños', 'Christopher Dean Cain', 'Padres', 'Roger Tanaka', 'Familiares', 'Roger Cain(Sibling)', 'Otras obras', 'Appeared as a motorcycle rider in George Strait\'s music video of his song "Carrying Your Love With Me"', 'Listings de publicidad', '3 entrevistas7 artículos3 ilustraciones25 fotos de portada de revista']
---
['Conocido/a por:', "God's Not Dead", '4,6', 'Marc Shelley', '2014', 'A contrarreloj', '6,5', 'Chris Harrison', '2003', 'Ratas a la carrera', '6,5', 'Shawn Kent', '2001', 'Lois y Clark: Las nuevas aventuras de Superman', '6,7', 'Serie de TV', 'BarryBill RobertsonClark Kent', '1993–1997 • 87 episodios']
---
['Créditos', 'Editar', 'Reparto209', 'Producción17', 'Guion6', 'Dirección3', 'Banda sonora1'

In [58]:
actor_datos = []
for dato in tupla_actor:
    

    patron_id = r'/name/(\w+)/'
    patron_nacimiento = re.compile(r'^(19|20)\d{2}$')
    
    if 'https' in dato:
        resultado = re.search(patron_id, dato)
        id = resultado.group(1)
        print(id)
        actor_datos.append(id)
    
       
    for indice, lista in enumerate(dato):
        #if 'Datos personales' in dato:
                      
        
        if 'Conocido/a por:' in dato:
            print(dato[indice +1 : indice + 16:2])
            actor_datos.append(dato[indice +1 : indice + 16:2])

            
        
        break
        


    

nm0001002
["God's Not Dead", 'Marc Shelley', 'A contrarreloj', 'Chris Harrison', 'Ratas a la carrera', 'Shawn Kent', 'Lois y Clark: Las nuevas aventuras de Superman', 'Serie de TV']


In [59]:
actor_datos

['nm0001002',
 ["God's Not Dead",
  'Marc Shelley',
  'A contrarreloj',
  'Chris Harrison',
  'Ratas a la carrera',
  'Shawn Kent',
  'Lois y Clark: Las nuevas aventuras de Superman',
  'Serie de TV']]

In [ ]:
if 'Datos personales' in dato:
            resultado = re.search(patron_nacimiento, tupla_actor[indice +1])
            año = resultado.group(1)
            print(año)

API Database - id actores

In [60]:
import requests


for id in lista_id:
	url = f"https://moviesdatabase.p.rapidapi.com/actors/{id}"

	headers = {
		"X-RapidAPI-Key": "902358e2e8msh3d16b2ef47d6b52p1fc5dajsnb0a5bd959614",
		"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers)

	print(response.json())

{'results': {'nconst': 'nm0001002', 'primaryName': 'Dean Cain', 'birthYear': 1966, 'deathYear': '\\N', 'primaryProfession': 'actor,producer,writer', 'knownForTitles': 'tt0106057,tt0250687,tt0313443,tt2528814'}}


In [61]:
response.json()

{'results': {'nconst': 'nm0001002',
  'primaryName': 'Dean Cain',
  'birthYear': 1966,
  'deathYear': '\\N',
  'primaryProfession': 'actor,producer,writer',
  'knownForTitles': 'tt0106057,tt0250687,tt0313443,tt2528814'}}

## ACTORES - ID Y LLAMADA API

In [ ]:
with open ('AccionDrama.json', 'r') as fichero:
    busqueda = json.load(fichero)

In [ ]:
lista_url = []

In [ ]:
for pelicula in busqueda:
    lista_url.append(f'https://www.imdb.com/title/{pelicula[0]}/fullcredits/?ref_=tt_ql_1')

In [82]:
lista_peliculas_y_actores = []
lista_todos_id = []

In [79]:
for url in lista_url:
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    # cookies
    try:
        driver.find_element('css selector', '#imdb-bmo-footer > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
    except:
        pass

    sleep(1)

    id_actores_por_peli = [url]
    lista_href = []

    for actor in range (2, 5):
        xpath_actor = driver.find_element('xpath', f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{actor}]/td[2]/a')
        href_actor = xpath_actor.get_attribute('href')
        print(f'Voy por href: {href_actor}')
        lista_href.append(href_actor)

    for href in lista_href:
        patron_id = r'/name/(\w+)/'
        if 'https' in href:
            resultado = re.search(patron_id, href)
            id = resultado.group(1)
            print(f'Voy por id: {id}')
            id_actores_por_peli.append(id)
            lista_todos_id.append(id)
    
    driver.close()        
    
    lista_peliculas_y_actores.append(id_actores_por_peli)


Voy por href: https://www.imdb.com/name/nm0001002/?ref_=ttfc_fc_cl_t1
Voy por href: https://www.imdb.com/name/nm0001299/?ref_=ttfc_fc_cl_t2
Voy por href: https://www.imdb.com/name/nm0923529/?ref_=ttfc_fc_cl_t3
Voy por id: nm0001002
Voy por id: nm0001299
Voy por id: nm0923529


In [81]:
id_actores_por_peli

['https://www.imdb.com/title/tt0113092/fullcredits/?ref_=tt_ql_1',
 'nm0001002',
 'nm0001299',
 'nm0923529']

dean: //*[@id="fullcredits_content"]/table[3]/tbody/tr[2]/td[2]/a
thomas: //*[@id="fullcredits_content"]/table[3]/tbody/tr[3]/td[2]/a
justin: //*[@id="fullcredits_content"]/table[3]/tbody/tr[4]/td[2]/a